# Introduction to NLP fundamentals

Common NLP problem is sequence to sequence (seq2seq).

In [1]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

## Visualizing a text dataset

In [2]:
# We want to predict whether the data is a disaster or not a disaster

In [3]:
import pandas as pd

train_df = pd.read_csv("../Datasets/nlp_getting_started/train.csv")
test_df = pd.read_csv("../Datasets/nlp_getting_started/test.csv")

In [4]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
# Shuffle our traininig dataframe

train_df_shuffled = train_df.sample(frac=1, random_state=42) #frac is what percentage do we want to shuffle. 1=100%
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [7]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [8]:
len(train_df), len(test_df)

(7613, 3263)

In [9]:
#Let's visualize some random training examples
import random
random_index = random.randint(0,len(train_df)-5)
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
    _, text, target = row # "_" -> to get rid of index
    print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
    print(f"Text:\n {text} \n")
    print("---\n")

Target: 0 (not real disaster)
Text:
 Ashes 4th Test: 10 Hilarious Twitter Reactions to Australia's collapse http://t.co/6DznEjuVD3 by @Absolut_Sumya15 

---

Target: 0 (not real disaster)
Text:
 'How many men would a human hew if a human could hew men?'

-popular tongue twister among woodchucks 

---

Target: 1 (real disaster)
Text:
 Families to sue over Legionnaires: More than 40 families affected by the fatal outbreak of Legionnaires' disea... http://t.co/8lCbDW7m2z 

---

Target: 0 (not real disaster)
Text:
 Reddit Will Now Quarantine Offensive Content http://t.co/unNx71v8qc 

---

Target: 1 (real disaster)
Text:
 Greer man dies in wreck http://t.co/n2qZbMZuly 

---



### Split data into Training and Validation set

In [10]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size = 0.1, #10% for validation set
                                                                            random_state = 42)

In [11]:
# Check lengths
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

(6851, 762, 6851, 762)

In [12]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting Text into Numbers

### Tokenization
Direct mapping of token (word or character) to a number

In [13]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [14]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

# Default parameters:

text_vectorizer = TextVectorization(max_tokens= None, # how many words in the vocabulary
                                    standardize= "lower_and_strip_punctuation",
                                    split= "whitespace",
                                    ngrams= None,
                                    output_mode = "int",
                                    output_sequence_length = None,
                                    pad_to_max_tokens = False)

2024-09-17 19:01:53.471175: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-09-17 19:01:53.471206: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-09-17 19:01:53.471215: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-09-17 19:01:53.472570: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-09-17 19:01:53.472591: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [15]:
# Find the average number of tokens (words) in the training tweets

round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [16]:
# Setup text vectorization variables

max_vocab_length = 10000
max_length = 15 # How many words of a tweet does a model see

text_vectorizer = TextVectorization(max_tokens= max_vocab_length,
                                    output_mode = "int",
                                    output_sequence_length = max_length)

In [17]:
# Fit the TextVectorizer to the training text

text_vectorizer.adapt(train_sentences)

In [18]:
# Create a sample sentence and tokenize it
sample_sentece = "There's a flood in my street!"
text_vectorizer([sample_sentece])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [19]:
# Choose a random sentence from the training set

random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
      \n\n Vectorized version:")
text_vectorizer(random_sentence)

Original text:
 #weed news How marijuana is making California drought worse - Christian Science Monitor http://t.co/2SZ7oKjRXi      

 Vectorized version:


<tf.Tensor: shape=(15,), dtype=int64, numpy=
array([2230,   58,   62, 3655,    9,  572,   90,  377, 1100,  861, 1770,
       5099,    1,    0,    0])>

In [20]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]

In [21]:
top_5_words, bottom_5_words

(['', '[UNK]', 'the', 'a', 'in'],
 ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1'])

In [22]:
# This way of using TextVectorization is a naive way. Let's try a better way

## Creating an Embedding using the Embedding Layer

In [23]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim = max_vocab_length,
                            output_dim = 128, 
                            #  input_length = max_length # deprecated
                            )
embedding

<Embedding name=embedding, built=False>

In [24]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence} \
    \n\nEmbedded version:")

# Embed the random sentence (turn it into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer(random_sentence))
sample_embed

Original text:
 @laevantine Fortunately I reworked the plumbing on my emergency chemical shower to draw from the glitter pipe for just such an occasion     

Embedded version:


<tf.Tensor: shape=(15, 128), dtype=float32, numpy=
array([[-0.01573881, -0.02428048, -0.03774982, ...,  0.02924741,
         0.04151918,  0.00101794],
       [ 0.01753597,  0.03842957,  0.00416219, ...,  0.01086243,
        -0.04905957, -0.03480424],
       [-0.03524651,  0.02210796,  0.04593063, ...,  0.01594783,
         0.01916618, -0.00476002],
       ...,
       [-0.00808883,  0.03384116,  0.02716379, ..., -0.00920238,
         0.04512927, -0.00204618],
       [-0.02249087, -0.02363896,  0.04514131, ..., -0.0444002 ,
        -0.03913986,  0.0405401 ],
       [-0.0414868 ,  0.00390953,  0.03453083, ...,  0.04156251,
        -0.02434552,  0.00698524]], dtype=float32)>

In [25]:
# Check out a single token's embedding

sample_embed[0], sample_embed[0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-1.5738808e-02, -2.4280477e-02, -3.7749816e-02, -4.1575909e-02,
         3.5279583e-02, -3.3267044e-02, -2.3366177e-02, -4.2547490e-02,
        -1.4964826e-03,  2.9451028e-03,  2.7149688e-02, -4.1196346e-02,
        -4.9776148e-02,  2.0335767e-02, -2.2826731e-02, -2.7889157e-02,
        -2.2223020e-02,  3.9984100e-03,  4.6430342e-03,  4.9823429e-02,
        -4.5904696e-02, -4.6097923e-02,  4.4565689e-02,  4.0892456e-02,
        -3.2361522e-02,  1.3091076e-02, -3.3051968e-02, -1.1448264e-03,
        -1.1316054e-03, -2.5923813e-02,  4.9265947e-02, -3.6813367e-02,
        -8.2686767e-03, -3.0828143e-02, -4.1895770e-02, -1.6561318e-02,
        -3.4894802e-02,  3.9492045e-02,  2.0356145e-02,  7.9297423e-03,
         2.1278154e-02,  1.8180337e-02, -1.3733529e-02,  2.0643663e-02,
        -2.7243614e-02, -3.8836014e-02,  4.0196028e-02,  3.7399832e-02,
         1.6221609e-02, -4.6263315e-02, -3.1748250e-02, -4.9358476e-02,
         3.9615

## Modelling a text dataset (running a series of experiments)

### Model 0: Getting a Baseline -> SKLearn's Naive Bayes using the TF-IDF formula.

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling Pipeline
model_0 = Pipeline([    ("tfidf", TfidfVectorizer()), # convert words to numbers using TFIDF
                    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [27]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of 79.27%


In [28]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [29]:
train_labels

array([0, 0, 1, ..., 1, 1, 0])

### Creating a function to evaluation our Model's experiments
* Accuracy
* Precision
* Recall
* F1-score

In [30]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
    """
    Calculates the model's accuracy, precions, recall, and f1 score of a binary classification model.
    """
    
    # Calculate model accuracy:
    model_accuracy = accuracy_score(y_true, y_pred)
    # Calculate the other 3:
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
    model_results = {"accuracy": model_accuracy,
                     "precision": model_precision,
                     "recall": model_recall,
                     "f1": model_f1}
    return model_results

In [32]:
# Get baseline results:
baseline_results = calculate_results(y_true=val_labels,y_pred=baseline_preds)
baseline_results

{'accuracy': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 0: Feed-forward NN Dense Model

In [68]:
# Create a tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create a directory to save TensorBoar logs
SAVE_DIR = "NLP_models_logs/model_logs"

In [69]:
# Build model with the Functional API
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string) # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn text into numbers
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x) # reduce the extra dimension given my the embedding
outputs = layers.Dense(1, activation="sigmoid")(x) # Sigmoid activation for binary outputs
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [70]:
model_1.summary()

Model: "model_1_dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_1      │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,280,129 (4.88 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

In [71]:
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [72]:
model_1_history = model_1.fit(x = train_sentences,
                              y = train_labels,
                              epochs = 5,
                              validation_data = (val_sentences,val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR, experiment_name="model_1_dense")]
                              )

Saving TensorBoard log files to: NLP_models_logs/model_logs/model_1_dense/20240917-202146
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.7060 - loss: 0.5990 - val_accuracy: 0.7848 - val_loss: 0.4937
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.8860 - loss: 0.3601 - val_accuracy: 0.7887 - val_loss: 0.4573
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9114 - loss: 0.2685 - val_accuracy: 0.7927 - val_loss: 0.4598
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9197 - loss: 0.2277 - val_accuracy: 0.7861 - val_loss: 0.4743
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9338 - loss: 0.1934 - val_accuracy: 0.7874 - val_loss: 0.4931


In [73]:
# Check the results

model_1.evaluate(val_sentences,val_labels)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7837 - loss: 0.5290


[0.49312764406204224, 0.787401556968689]

In [74]:
# Make predictions and evaluate our results
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


array([[0.38896105],
       [0.8011162 ],
       [0.9986345 ],
       [0.09726495],
       [0.08725371],
       [0.9529148 ],
       [0.9184859 ],
       [0.9967526 ],
       [0.9777753 ],
       [0.25817516]], dtype=float32)

In [75]:
# Convert model predictions probabilities to label format

model_1_preds = tf.squeeze(tf.round(model_1_pred_probs[:]))
model_1_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [76]:
# Calculate our model 1's results

model_1_results = calculate_results(y_true= val_labels, y_pred=model_1_preds)
model_1_results

{'accuracy': 0.7874015748031497,
 'precision': 0.790739069704199,
 'recall': 0.7874015748031497,
 'f1': 0.7849686345949081}

In [77]:
baseline_results

{'accuracy': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [78]:
import numpy as np
np.array(list(baseline_results.values())) > np.array(list(model_1_results.values()))

array([ True,  True,  True,  True])

In [79]:
# Model 1 did not perform better than our simple baseline.